In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import logging
from tqdm import tqdm
from skimage.measure import label
import sys 
import random 
import torch.backends.cudnn as cudnn
# torch 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# Module 
from dataset.basedataset import ACDCDataset
from dataset.utils import RandomGenerator
from utils.losses import DiceLoss
from networks.net_factory import BCP_net
from utils import masks
from utils.valid2d import test_single_volume, test_single_volume_mean
from networks.utils import save_net_opt, load_net, load_net_opt
from networks.utils import update_model_ema

# writer 
from torch.utils.tensorboard import SummaryWriter

2025-04-11 15:01:26.503894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-11 15:01:26.517046: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-11 15:01:26.520931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 15:01:26.532812: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-11 15:01:27.530151: W tensorflow/compiler/tf2

In [2]:
import warnings 
warnings.simplefilter('always')
warnings.filterwarnings('default')

In [3]:
# param 
from utils.params import params
args = params() 

In [4]:
writer = SummaryWriter(log_dir= 'modelSDCL') 

In [5]:
def get_ACDC_LargestCC(segmentation):
    class_list = []
    for i in range(1, 4):
        temp_prob = segmentation == i * torch.ones_like(segmentation)
        temp_prob = temp_prob.detach().cpu().numpy()
        labels = label(temp_prob)
        # -- with 'try'
        assert (labels.max() != 0)  # assume at least 1 CC
        largestCC = labels == np.argmax(np.bincount(labels.flat)[1:]) + 1
        class_list.append(largestCC * i)
    acdc_largestCC = class_list[0] + class_list[1] + class_list[2]
    return torch.from_numpy(acdc_largestCC).cuda()


def get_ACDC_2DLargestCC(segmentation):
    batch_list = []
    N = segmentation.shape[0]
    for i in range(0, N):
        class_list = []
        for c in range(1, 4):
            temp_seg = segmentation[i]  # == c *  torch.ones_like(segmentation[i])
            temp_prob = torch.zeros_like(temp_seg)
            temp_prob[temp_seg == c] = 1
            temp_prob = temp_prob.detach().cpu().numpy()
            labels = label(temp_prob)
            if labels.max() != 0:
                largestCC = labels == np.argmax(np.bincount(labels.flat)[1:]) + 1
                class_list.append(largestCC * c)
            else:
                class_list.append(temp_prob)

        n_batch = class_list[0] + class_list[1] + class_list[2]
        batch_list.append(n_batch)
    batch_np = np.array(batch_list)
    batch_tensor = torch.from_numpy(batch_np).float().cuda()
    return batch_tensor 


def get_ACDC_masks(output, nms=0):
    probs = F.softmax(output, dim=1)
    _, probs = torch.max(probs, dim=1)
    if nms == 1:
        probs = get_ACDC_2DLargestCC(probs)
    return probs

In [6]:
dice_loss = DiceLoss(n_classes= 4)
def mix_loss(output, img_l, patch_l, mask, l_weight=1.0, u_weight=0.5, unlab=False):
    CE = nn.CrossEntropyLoss(reduction='none')
    img_l, patch_l = img_l.type(torch.int64), patch_l.type(torch.int64)
    output_soft = F.softmax(output, dim=1)
    image_weight, patch_weight = l_weight, u_weight
    if unlab:
        image_weight, patch_weight = u_weight, l_weight
    patch_mask = 1 - mask

    loss_dice = dice_loss(output_soft, img_l.unsqueeze(1), mask.unsqueeze(1)) * image_weight
    loss_dice += dice_loss(output_soft, patch_l.unsqueeze(1), patch_mask.unsqueeze(1)) * patch_weight
    loss_ce = image_weight * (CE(output, img_l) * mask).sum() / (mask.sum() + 1e-16)
    loss_ce += patch_weight * (CE(output, patch_l) * patch_mask).sum() / (patch_mask.sum() + 1e-16)  # loss = loss_ce
    return loss_dice, loss_ce


In [7]:
def pre_train(args, snapshot_path): 
    writer.add_text('Phase','========================== Pretrain =============================', 0)
    # Pretrain
    num_clases = args.num_classes
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

    # Load data 
    db_val = ACDCDataset(base_dir= args.root_dir, split= 'val')

    c_batch_size = 12 
    trainset_lab_a = ACDCDataset(base_dir= args.root_dir, 
                                split= 'train_lab', 
                                transform= transforms.Compose([RandomGenerator(args.patch_size)]))

    trainset_lab_b = ACDCDataset(base_dir= args.root_dir, 
                            split= 'train_lab',
                            reverse= True, 
                            transform= transforms.Compose([RandomGenerator(args.patch_size)]))

    train_loader_a = DataLoader(trainset_lab_a, batch_size= c_batch_size, shuffle= False, num_workers= 0, drop_last= True)
    train_loader_b = DataLoader(trainset_lab_b, batch_size= c_batch_size, shuffle=False,num_workers= 0, drop_last= True)
    val_loader = DataLoader(db_val, batch_size=1, shuffle= False, num_workers=0)


    # Network
    model1 = BCP_net(model= 'unet', in_chns=1, num_classes= num_clases)
    model2 = BCP_net(model= 'ResUnet', in_chns=1, num_classes= num_clases) 
    optimizer1 = optim.Adam(model1.parameters(), lr= 1e-3)
    optimizer2 = optim.Adam(model2.parameters(), lr= 1e-3)
    logging.info('optim.Adam pre_train problem')

    logging.info('Start pre-traininig')
    logging.info(f'{len(train_loader_a)} iterations per epoch ')

    model1.train() 
    model2.train() 

    iter_num = 0 
    best_performance1 = 0.0 
    best_performance2 = 0.0 
    max_epoch = args.max_pretrain_epoch
    iterator = tqdm(range(1, max_epoch), ncols= 70)
    for epoch in iterator: 
        logging.info('\n')
        for step, ((img_a, lab_a), (img_b, lab_b)) in enumerate(zip(train_loader_a, train_loader_b)): 
            img_a, img_b, lab_a, lab_b = img_a.cuda(), img_b.cuda(), lab_a.cuda(), lab_b.cuda() 

            img_mask, loss_mask = masks.generate_mask(img_a) 

            # -- original 
            net_input = img_a * img_mask + img_b * (1- img_mask) 
            
            out_mixl_1 = model1(net_input) 
            out_mixl_2 = model2(net_input)

            # print(f'Input.shape = {net_input.shape}')
            # print(f'Label_a.shape = {lab_a.shape}')
            # print(f'Label_b.shape = {lab_b.shape}')
            # print(f'Output.shape = {out_mixl_1.shape}')
            loss_dice_1, loss_ce_1 = mix_loss(out_mixl_1, lab_a, lab_b, loss_mask, u_weight= 1.0, unlab= True)
            loss_dice_2, loss_ce_2 = mix_loss(out_mixl_2, lab_a, lab_b,loss_mask, u_weight= 1.0, unlab= True) 

            loss1 = (loss_dice_1 + loss_ce_1) / 2 
            loss2 = (loss_dice_2 + loss_ce_2) / 2 

            optimizer1.zero_grad() 
            loss1.backward() 
            optimizer1.step() 

            optimizer2.zero_grad() 
            loss2.backward() 
            optimizer2.step() 

            iter_num += 1
            writer.add_scalar('Loss/model1_total', loss1.item(), iter_num) 
            writer.add_scalar('Loss/model1_dice', loss_dice_1.item(), iter_num)
            writer.add_scalar('Loss/model1_ce', loss_ce_1.item(), iter_num)
            writer.add_scalar('Loss/model2_total', loss2.item(), iter_num)
            writer.add_scalar('Loss/model2_dice', loss_dice_2.item(), iter_num)
            writer.add_scalar('Loss/model2_ce', loss_ce_2.item(), iter_num)
            logging.info(f'iterations: {iter_num} loss: {loss1} loss_dice: {loss_dice_1} loss_ce: {loss_ce_1}')
        
        if epoch >= 0 and epoch % 5 == 0: 
            model1.eval() 
            model2.eval() 
            metric_list_1 = 0.0 
            metric_list_2 = 0.0 
            for _, (img_val, lab_val) in tqdm(enumerate(val_loader), ncols= 70): 
                metric_i_1 = test_single_volume(img_val, lab_val, model1, classes= num_clases)
                metric_i_2 = test_single_volume(img_val, lab_val, model2, classes= num_clases)

                metric_list_1 += np.array(metric_i_1)
                metric_list_2 += np.array(metric_i_2)
            
            metric_list_1 = metric_list_1 / len(db_val)
            metric_list_2 = metric_list_2 / len(db_val)

            performance_1 = np.mean(metric_list_1, axis= 0)[0] 
            performance_2 = np.mean(metric_list_2, axis= 0)[0] 

            if performance_1 > best_performance1: 
                best_performance1 = performance_1 

                save_model_path = os.path.join(snapshot_path, f"iter_{iter_num}_dice_{performance_1:.4f}.pth")
                save_best_path = os.path.join(snapshot_path, 'best_model_1.pth')  

                save_net_opt(model1, optimizer1, save_model_path)          
                save_net_opt(model1, optimizer1, save_best_path)
            
            if performance_2 > best_performance2: 
                best_performance2 = performance_2 

                save_model_path = os.path.join(snapshot_path, f"iter_{iter_num}_dice_{performance_2:.4f}.pth")
                save_best_path = os.path.join(snapshot_path, 'best_model_2.pth')  

                save_net_opt(model2, optimizer2, save_model_path)          
                save_net_opt(model2, optimizer2, save_best_path)
            writer.add_scalar('Performance/UNet', performance_1.item(), iter_num)
            writer.add_scalar('Performance/ResUnet', performance_2.item(), iter_num)
            logging.info('iteration %d : mean_dice : %f, val_maxdice : %f' % (iter_num, performance_1, best_performance1))
            logging.info('resnet iteration %d : mean_dice : %f, val_maxdice : %f' % (iter_num, performance_2, best_performance2))
            model1.train()
            model2.train() 
                

In [8]:
# Support self-train ----- NEED TO UNDERSTAND AGAIN !!!!!  
def softmax_mse_loss(input_logits, target_logits):
    """Takes softmax on both sides and returns MSE loss
    Note:
    - Returns the sum over all examples. Divide by the batch size afterwards
      if you want the mean.
    - Sends gradients to inputs but not the targets.
    """
    assert input_logits.size() == target_logits.size()
    input_softmax = F.softmax(input_logits, dim=1)
    # target_softmax = F.softmax(target_logits, dim=1)
    mse_loss = (input_softmax - target_logits) ** 2
    return mse_loss

def to_one_hot(tensor, nClasses):
    """ Input tensor : Nx1xHxW
    :param tensor:
    :param nClasses:
    :return:
    """
    assert tensor.max().item() < nClasses, 'one hot tensor.max() = {} < {}'.format(torch.max(tensor), nClasses)
    assert tensor.min().item() >= 0, 'one hot tensor.min() = {} < {}'.format(tensor.min(), 0)

    size = list(tensor.size())
    assert size[1] == 1
    size[1] = nClasses
    one_hot = torch.zeros(*size)
    if tensor.is_cuda:
        one_hot = one_hot.cuda(tensor.device)
    one_hot = one_hot.scatter_(1, tensor, 1)
    return one_hot

def get_XOR_region(mixout1, mixout2): 
    s1 = torch.softmax(mixout1, dim=1)
    l1 = torch.argmax(s1, dim=1) 
    s2 = torch.softmax(mixout2, dim=1) 
    l2 = torch.argmax(s2, dim= 1) 

    diff_mask = (l1 != l2) 
    return diff_mask

def mix_mse_loss(net3_output, img_l, patch_l, mask= None, l_weight=1.0, u_weight=0.5, unlab= False, diff_mask=None): 
    img_l, patch_l = img_l.type(torch.int64),patch_l.type(torch.int64) 
    image_weight, patch_weight = l_weight, u_weight
    if unlab: 
        image_weight, patch_weight = u_weight, l_weight

    patch_mask = 1 - mask 
    img_l_onehot = to_one_hot(img_l.unsqueeze(1), 4)
    patch_l_onehot = to_one_hot(patch_l.unsqueeze(1), 4)

    mse_loss = torch.mean(softmax_mse_loss(net3_output,img_l_onehot), dim=1)*mask * image_weight
    mse_loss += torch.mean(softmax_mse_loss(net3_output, patch_l_onehot), dim= 1) * patch_mask * patch_weight

    loss = torch.sum(diff_mask * mse_loss) / (torch.sum(diff_mask) + 1e-16)
    return loss 
     
voxel_kl_loss = nn.KLDivLoss(reduction='none')
def mix_max_kl_loss(net3_output, img_l, patch_l, mask, l_weight=1.0, u_weight=0.5, unlab=False, diff_mask=None):
    img_l, patch_l = img_l.type(torch.int64), patch_l.type(torch.int64)

    image_weight, patch_weight = l_weight, u_weight
    if unlab:
        image_weight, patch_weight = u_weight, l_weight

    patch_mask = 1 - mask

    with torch.no_grad():
        s1 = torch.softmax(net3_output, dim=1)
        l1 = torch.argmax(s1, dim=1)
        img_diff_mask = (l1 != img_l)
        patch_diff_mask = (l1 != patch_l)

        uniform_distri = torch.ones(net3_output.shape)
        uniform_distri = uniform_distri.cuda()

    kl_loss = torch.mean(voxel_kl_loss(F.log_softmax(net3_output, dim=1), uniform_distri),
                         dim=1) * mask * img_diff_mask * image_weight
    kl_loss += torch.mean(voxel_kl_loss(F.log_softmax(net3_output, dim=1), uniform_distri),
                          dim=1) * patch_mask * patch_diff_mask * patch_weight

    sum_diff = torch.sum(mask * img_diff_mask * diff_mask) + torch.sum(patch_mask * patch_diff_mask * diff_mask)
    loss = torch.sum(diff_mask * kl_loss) / (sum_diff + 1e-16)
    return loss

In [9]:
# self-train 
def selftrain(args, pre_snapshot_path, snapshot_path): 
    num_classes = args.num_classes
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

    # Load data 
    db_val = ACDCDataset(base_dir= args.root_dir, split= 'val')
    c_batch_size = 12 
    trainset_lab_a = ACDCDataset(base_dir= args.root_dir, 
                                split= 'train_lab', 
                                transform= transforms.Compose([RandomGenerator(args.patch_size)]))
    trainset_unlab_a = ACDCDataset(base_dir= args.root_dir, 
                                split= 'train_unlab', 
                                transform= transforms.Compose([RandomGenerator(args.patch_size)]))

    trainset_lab_b = ACDCDataset(base_dir= args.root_dir, 
                                split= 'train_lab', 
                                reverse= True, 
                                transform= transforms.Compose([RandomGenerator(args.patch_size)]))
    trainset_unlabel_b = ACDCDataset(base_dir= args.root_dir, 
                                    split= 'train_unlab', 
                                    reverse= True, 
                                    transform= transforms.Compose([RandomGenerator(args.patch_size)]))

    lab_loader_a = DataLoader(trainset_lab_a, batch_size= c_batch_size, shuffle= False, num_workers= 0, drop_last= True)
    lab_loader_b = DataLoader(trainset_lab_b, batch_size= c_batch_size, shuffle= False, num_workers= 0, drop_last= True) 
    unlab_loader_a = DataLoader(trainset_unlab_a, batch_size= c_batch_size, shuffle= False, num_workers=0, drop_last= True) 
    unlab_loader_b = DataLoader(trainset_unlabel_b, batch_size= c_batch_size, shuffle= False, num_workers=0, drop_last= True) 
    valloader = DataLoader(db_val, batch_size=1, shuffle= False, num_workers=1)

    # Prepare model
    pre_train_model1 = os.path.join(pre_snapshot_path, 'best_model_1.pth')
    pre_train_model2 = os.path.join(pre_snapshot_path, 'best_model_2.pth')

    model1 = BCP_net(model= 'unet', in_chns=1, num_classes= num_classes)
    model2 = BCP_net(model= 'ResUnet', in_chns= 1, num_classes= num_classes)
    ema_model = BCP_net(model= 'unet', in_chns=1, num_classes= num_classes, ema= True)

    optimizer1 = optim.Adam(model1.parameters(), lr= 1e-3)
    optimizer2 = optim.Adam(model2.parameters(), lr= 1e-3)

    load_net_opt(ema_model, optimizer1, pre_train_model1)
    load_net_opt(model1, optimizer1, pre_train_model1)
    load_net_opt(model2, optimizer2, pre_train_model2)
    print(f'Load from: {pre_train_model1}')
    logging.info(f'Load from {pre_snapshot_path}')

    logging.info('Start self-training')
    model1.train() 
    model2.train() 
    ema_model.train() 

    iter_num = 0 
    best_performance_1 = 0.0 
    best_performance_2 = 0.0 
    best_performance_mean = 0.0 

    max_epoch = args.max_seftrain_epoch 
    iterator = tqdm(range(1, max_epoch), ncols= 70) 
    print(f'Trainloader LAB: {len(lab_loader_a)}')
    print(f'Trainloader UNLAB: {len(unlab_loader_a)}')
    for epoch in iterator: 
        for step, ((img_a, lab_a), (img_b, lab_b), (unimg_a, unlab_a), (unimg_b, unlab_b)) in enumerate(zip(lab_loader_a, lab_loader_b, unlab_loader_a, unlab_loader_b)):
            img_a, lab_a, img_b, lab_b, unimg_a, unlab_a, unimg_b, unlab_b = img_a.cuda(), lab_a.cuda(), img_b.cuda(), lab_b.cuda(), unimg_a.cuda(), unlab_a.cuda(), unimg_b.cuda(), unlab_b.cuda()

            # Pseudo label        
            with torch.no_grad(): 
                pre_a = ema_model(unimg_a)
                pre_b = ema_model(unimg_b)
                plab_a = get_ACDC_masks(pre_a, nms=1)
                plab_b = get_ACDC_masks(pre_b, nms=1)

                img_mask, loss_mask = masks.generate_mask(img_a)
            
            # BCP problem 
            net_input_l = unimg_a * img_mask + img_b*(1- img_mask)
            net_input_unl = img_a * img_mask + unimg_b * ( 1 - img_mask)

            out_l_1 =  model1(net_input_l)
            out_unl_1 = model1(net_input_unl)
            out_l_2 = model2(net_input_l)
            out_unl_2 = model2((net_input_unl))
            # TODO: CONSIDER PROBLEM HERE 
            l_dice_1, l_ce_1 = mix_loss(out_l_1, plab_a.long(), lab_b, loss_mask, u_weight= args.u_weight, unlab= True) 
            un_dice_1, un_ce_1 = mix_loss(out_unl_1, lab_a, plab_b.long(), loss_mask, u_weight= args.u_weight)

            l_dice_2, l_ce_2 = mix_loss(out_l_2, plab_a.long(), lab_b, loss_mask, u_weight= args.u_weight, unlab= True) 
            un_dice_2, un_ce_2 = mix_loss(out_unl_2, lab_a, plab_b.long(), loss_mask, u_weight= args.u_weight) 
            
            # Average loss 
            loss_dice_1 = l_dice_1 + un_dice_1
            loss_ce_1 = l_ce_1 + un_ce_1
            loss_dice_2 = l_dice_2 + un_dice_2 
            loss_ce_2 = l_ce_2 + un_ce_2  

            with torch.no_grad(): 
                diff_mask1 = get_XOR_region(out_l_1, out_l_2)
                diff_mask_2 = get_XOR_region(out_unl_1, out_unl_2)
            
            net1_mse_loss_lab = mix_mse_loss(out_l_1, plab_a.long(), lab_b, loss_mask, unlab= True, diff_mask= diff_mask1)
            net1_kl_loss_lab = mix_max_kl_loss(out_l_1, plab_a.long(), lab_b, loss_mask, unlab= True, diff_mask= diff_mask1)

            net1_mse_loss_unlab = mix_mse_loss(out_unl_1, lab_a, plab_b.long(), loss_mask, diff_mask= diff_mask_2)
            net1_kl_loss_unlab = mix_max_kl_loss(out_unl_1, lab_a, plab_b.long(), loss_mask, diff_mask= diff_mask_2)

            net2_mse_loss_lab = mix_mse_loss(out_l_2, plab_a.long(), lab_b, loss_mask, unlab= True, diff_mask= diff_mask1)
            net2_kl_loss_lab = mix_max_kl_loss(out_l_2, lab_a, plab_b.long(), loss_mask, unlab= True, diff_mask= diff_mask1)

            net2_mse_loss_unlab = mix_mse_loss(out_unl_2, lab_a, plab_b.long(), loss_mask, diff_mask= diff_mask_2)
            net2_kl_loss_unlab = mix_max_kl_loss(out_unl_2, lab_a, plab_b.long(), loss_mask, diff_mask= diff_mask_2)

            loss_1 = (loss_dice_1 + loss_ce_1)/2 + 0.5 * (net1_mse_loss_lab + net1_mse_loss_unlab) + 0.05 * (net1_kl_loss_lab + net1_kl_loss_unlab)
            loss_2 = (loss_dice_2 + loss_ce_2)/2 + 0.5 * (net2_mse_loss_lab + net2_mse_loss_unlab) + 0.05 * (net2_kl_loss_lab + net2_kl_loss_unlab)

            optimizer1.zero_grad() 
            loss_1.backward()
            optimizer1.step() 

            optimizer2.zero_grad() 
            loss_2.backward() 
            optimizer2.step() 

            iter_num += 1 
            update_model_ema(model1, ema_model, 0.99)
            

            logging.info({
                'iter': iter_num,
                'mix_dice': loss_dice_1.item(),
                'mix_ce': loss_ce_1.item(),
                'mse_lab': net1_mse_loss_lab.item(),
                'mse_unlab': net1_mse_loss_unlab.item(),
                'kl_lab': net1_kl_loss_lab.item(),
                'kl_unlab': net1_kl_loss_unlab.item()
            })

            if iter_num % 200 == 0:
                with torch.no_grad(): 
                    model1.eval()
                    model2.eval()
                    metric_list = 0.0
                    metric_list_2 = 0.0
                    metric_list_mean = 0.0

                    for _, (img_val, lab_val) in tqdm(enumerate(valloader), ncols=70):
                        metric_i = test_single_volume(img_val, lab_val, model1, classes=num_classes)
                        metric_i_2 = test_single_volume(img_val, lab_val, model2, classes=num_classes)
                        metric_i_mean = test_single_volume_mean(img_val, lab_val, model1, model2, classes=num_classes)

                        metric_list += np.array(metric_i)
                        metric_list_2 += np.array(metric_i_2)
                        metric_list_mean += np.array(metric_i_mean)

                    metric_list = metric_list / len(db_val)
                    metric_list_2 = metric_list_2 / len(db_val)
                    metric_list_mean = metric_list_mean / len(db_val)

                    performance_1 = np.mean(metric_list, axis=0)[0]
                    performance_2 = np.mean(metric_list_2, axis=0)[0]
                    performance_mean = np.mean(metric_list_mean, axis=0)[0]

                    if performance_1 > best_performance_1:
                        best_performance_1 = performance_1
                        save_mode_path = os.path.join(snapshot_path,
                                                        'iter_{}_dice_{}.pth'.format(iter_num, round(best_performance_1, 4)))
                        save_best_path = os.path.join(snapshot_path, 'best_model.pth')
                        save_net_opt(model1, optimizer1, save_mode_path)
                        save_net_opt(model1, optimizer1, save_best_path)

                    if performance_2 > best_performance_2:
                        best_performance_2 = performance_2
                        save_mode_path = os.path.join(snapshot_path,
                                                        'iter_{}_dice_{}_res.pth'.format(iter_num, round(best_performance_2, 4)))
                        save_best_path = os.path.join(snapshot_path, 'best_model_res.pth')
                        save_net_opt(model2, optimizer2, save_mode_path)
                        save_net_opt(model2, optimizer2, save_best_path)

                    if performance_mean > best_performance_mean:
                        best_performance_mean = performance_mean

                        save_mode_path1 = os.path.join(snapshot_path, 'iter_{}_dice_{}_v.pth'.format(iter_num, round(
                            best_performance_mean, 4)))
                        save_best_path1 = os.path.join(snapshot_path, 'best_model_v.pth')

                        save_mode_path2 = os.path.join(snapshot_path, 'iter_{}_dice_{}_r.pth'.format(iter_num, round(
                            best_performance_mean, 4)))
                        save_best_path2 = os.path.join(snapshot_path, 'best_model_r.pth')

                        save_net_opt(model1, optimizer1, save_mode_path1)
                        save_net_opt(model1, optimizer1, save_best_path1)

                        save_net_opt(model2, optimizer2, save_mode_path2)
                        save_net_opt(model2, optimizer2, save_best_path2)


In [20]:
if args.deterministic:
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    # torch.backends.cudnn.benchmark = False
    # torch.backends.cudnn.deterministic = True
    cudnn.benchmark = False
    # cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)

pretrain_snapshot_path = 'modelSDCL/pretrain' 
selftrain_snapshot_path = 'modelSDCL/selftrain'
for snapshot_path in [pretrain_snapshot_path,selftrain_snapshot_path]: 
    if not os.path.exists(snapshot_path): 
        os.makedirs(snapshot_path)
        
logging.basicConfig(
    filename= 'modelSDCL/log.txt', 
    level= logging.INFO, 
    format= '[%(asctime)s.%(msecs)03d] %(message)s', 
    datefmt= '%H:%M:%S'

)
pre_train(args, pretrain_snapshot_path)

Mode: val: 20 samples in total
Mode: train_lab: 136 samples in total
Mode: train_lab: 136 samples in total


20it [00:12,  1.61it/s]                | 4/40 [03:22<30:27, 50.76s/it]
20it [00:12,  1.60it/s]                | 9/40 [07:50<26:44, 51.75s/it]
20it [00:13,  1.54it/s]               | 14/40 [12:20<22:45, 52.52s/it]
20it [00:12,  1.60it/s]               | 19/40 [16:51<18:25, 52.64s/it]
20it [00:12,  1.58it/s]█▊             | 24/40 [21:20<14:01, 52.62s/it]
20it [00:12,  1.57it/s]█████▉         | 29/40 [25:51<09:39, 52.71s/it]
20it [00:12,  1.55it/s]██████████     | 34/40 [30:23<05:17, 52.94s/it]
20it [00:12,  1.57it/s]██████████████▏| 39/40 [34:55<00:52, 52.82s/it]
100%|█████████████████████████████████| 40/40 [36:00<00:00, 54.00s/it]


In [10]:
if args.deterministic:
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    # torch.backends.cudnn.benchmark = False
    # torch.backends.cudnn.deterministic = True
    cudnn.benchmark = False
    # cudnn.deterministic = True
    torch.use_deterministic_algorithms(True)

pretrain_snapshot_path = 'modelSDCL/pretrain' 
selftrain_snapshot_path = 'modelSDCL/selftrain'
for snapshot_path in [pretrain_snapshot_path,selftrain_snapshot_path]: 
    if not os.path.exists(snapshot_path): 
        os.makedirs(snapshot_path)
        
logging.basicConfig(
    filename= 'modelSDCL/log.txt', 
    level= logging.INFO, 
    format= '[%(asctime)s.%(msecs)03d] %(message)s', 
    datefmt= '%H:%M:%S'

)
selftrain(args, pretrain_snapshot_path, selftrain_snapshot_path)

Mode: val: 20 samples in total
Mode: train_lab: 136 samples in total
Mode: train_unlab: 1176 samples in total
Mode: train_lab: 136 samples in total
Mode: train_unlab: 1176 samples in total


/home/lequocthinh/Desktop/pythonCode/SSL4MIS/BCP/Medical_Seg_Semi/networks/utils.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(str(path))


Load from: modelSDCL/pretrain/best_model_1.pth


  0%|                                          | 0/20 [00:00<?, ?it/s]

Trainloader LAB: 113
Trainloader UNLAB: 980


  5%|█▋                               | 1/20 [01:58<37:35, 118.71s/it]/home/lequocthinh/miniconda3/envs/DeepLearning/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=10269) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
20it [00:19,  1.01it/s]
20it [00:20,  1.01s/it]               | 3/20 [06:17<35:32, 125.42s/it]
20it [00:19,  1.00it/s]               | 5/20 [10:34<31:36, 126.41s/it]
20it [00:18,  1.10it/s]               | 7/20 [14:52<27:28, 126.81s/it]
20it [00:19,  1.03it/s]               | 8/20 [17:11<26:07, 130.58s/it]
 40%|█████████████▏                   | 8/20 [19:24<29:06, 145.55s/it]


KeyboardInterrupt: 